In [1]:
import numpy as np
import pandas as pd
import statsmodels.formula.api as sm
import datetime

In [4]:
# Updated sentiment score data
df_new=pd.read_pickle('/home/ec2-user/SageMaker/New Uncertainty/Jan1985-Dec2021/RegRelevant_ArticleSentimentWordsScores.pkl')
print(df_new.info())

print("Updated Start Dates:")
print(df_new.sort_values('StartDate')[['StartDate']])

<class 'pandas.core.frame.DataFrame'>
Int64Index: 990262 entries, 0 to 990261
Data columns (total 35 columns):
 #   Column                       Non-Null Count   Dtype         
---  ------                       --------------   -----         
 0   ID                           990262 non-null  object        
 1   Title                        990262 non-null  object        
 2   Type                         990262 non-null  object        
 3   StartDate                    990262 non-null  datetime64[ns]
 4   EndDate                      990262 non-null  object        
 5   TextWordCount                990262 non-null  object        
 6   PubTitle                     990262 non-null  object        
 7   SourceType                   990262 non-null  object        
 8   Year                         990262 non-null  float64       
 9   Month                        990262 non-null  float64       
 10  Newspaper                    990262 non-null  object        
 11  RegSentsExpand            

In [24]:
# # Append previous and updated data
# df=df_old.append(df_new).drop_duplicates('ID').sort_values(['StartDate','Newspaper'])
# print(df.info())
# print(df[['StartDate']])

In [5]:
# Reg relevant articles
df=df_new[df_new['RegRelevance']==1].sort_values(['StartDate','PubTitle'])
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 608172 entries, 0 to 990261
Data columns (total 35 columns):
 #   Column                       Non-Null Count   Dtype         
---  ------                       --------------   -----         
 0   ID                           608172 non-null  object        
 1   Title                        608172 non-null  object        
 2   Type                         608172 non-null  object        
 3   StartDate                    608172 non-null  datetime64[ns]
 4   EndDate                      608172 non-null  object        
 5   TextWordCount                608172 non-null  object        
 6   PubTitle                     608172 non-null  object        
 7   SourceType                   608172 non-null  object        
 8   Year                         608172 non-null  float64       
 9   Month                        608172 non-null  float64       
 10  Newspaper                    608172 non-null  object        
 11  RegSentsExpand            

In [29]:
# Change variable types
df['StartDate']=df['StartDate'].astype('datetime64[ns]')
df['Year']=df['StartDate'].dt.year
df['Month']=df['StartDate'].dt.month
df['Newspaper']=df['Newspaper'].astype('category')
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 608172 entries, 0 to 608171
Data columns (total 10 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   ID                608172 non-null  int64         
 1   StartDate         608172 non-null  datetime64[ns]
 2   Newspaper         608172 non-null  category      
 3   PubTitle          608172 non-null  object        
 4   UncertaintyScore  608172 non-null  float64       
 5   GIscore           608172 non-null  float64       
 6   LMscore           608172 non-null  float64       
 7   LSDscore          608172 non-null  float64       
 8   Year              608172 non-null  int64         
 9   Month             608172 non-null  int64         
dtypes: category(1), datetime64[ns](1), float64(4), int64(3), object(1)
memory usage: 47.0+ MB
None


In [32]:
# Check on publications considered
print(df['PubTitle'].value_counts())

Wall Street Journal                        99838
New York Times (Online)                    89010
New York Times                             75616
Wall Street Journal (Online)               69036
The Washington Post                        41440
Los Angeles Times                          36955
Los Angeles Times (pre-1997 Fulltext)      36798
Chicago Tribune                            28868
Boston Globe                               27113
Chicago Tribune (pre-1997 Fulltext)        23236
The Washington Post (pre-1997 Fulltext)    20734
Boston Globe (pre-1997 Fulltext)           13366
USA TODAY                                  11175
The Washington Post (Online)               10531
USA TODAY (pre-1997 Fulltext)               6079
USA Today (Online)                          5259
Los Angeles Times (Online)                  4685
Boston Globe (Online)                       4383
Chicago Tribune (Online)                    4050
Name: PubTitle, dtype: int64


In [6]:
# Specify index start date and end date
start_date=datetime.datetime(1985,1,1)
end_date=datetime.datetime(2021,12,31)
end_month=end_date.strftime('%b%Y')

df=df[(df['StartDate']>=start_date) & (df['StartDate']<=end_date)].sort_values('StartDate').reset_index(drop=True)
print(df[['StartDate','Year','Month']])

        StartDate  Year  Month
0      1985-01-01  1985      1
1      1985-01-01  1985      1
2      1985-01-01  1985      1
3      1985-01-01  1985      1
4      1985-01-01  1985      1
...           ...   ...    ...
608167 2021-12-31  2021     12
608168 2021-12-31  2021     12
608169 2021-12-31  2021     12
608170 2021-12-31  2021     12
608171 2021-12-31  2021     12

[608172 rows x 3 columns]


In [7]:
# Create a DF with all unique years and months
df_ym=df[['Year','Month']].drop_duplicates().reset_index(drop=True).reset_index()
df_ym['YM']=df_ym['index']+1
df_ym=df_ym.drop('index',axis=1)
print(df_ym)

     Year  Month   YM
0    1985      1    1
1    1985      2    2
2    1985      3    3
3    1985      4    4
4    1985      5    5
..    ...    ...  ...
439  2021      8  440
440  2021      9  441
441  2021     10  442
442  2021     11  443
443  2021     12  444

[444 rows x 3 columns]


In [8]:
df=df.merge(df_ym[['Year','Month','YM']],on=['Year','Month'],how='left').sort_values(['Year','Month']).reset_index(drop=True)
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 608172 entries, 0 to 608171
Data columns (total 10 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   ID                608172 non-null  int64         
 1   StartDate         608172 non-null  datetime64[ns]
 2   Newspaper         608172 non-null  category      
 3   UncertaintyScore  608172 non-null  float64       
 4   GIscore           608172 non-null  float64       
 5   LMscore           608172 non-null  float64       
 6   LSDscore          608172 non-null  float64       
 7   Year              608172 non-null  int64         
 8   Month             608172 non-null  int64         
 9   YM                608172 non-null  int64         
dtypes: category(1), datetime64[ns](1), float64(4), int64(4)
memory usage: 42.3 MB
None


In [9]:
print(df[['ID','UncertaintyScore','GIscore','LMscore','LSDscore']].head())

          ID  UncertaintyScore   GIscore   LMscore  LSDscore
0  294326637          0.000000  0.000000 -2.631579  0.000000
1  290805542          2.985075  0.746269 -1.492537  0.746269
2  292036407          0.000000  4.040404 -1.010101  1.010101
3  292041332          1.538462  0.000000  0.000000 -1.538462
4  292029630          0.000000  3.296703  0.000000  1.098901


In [10]:
print(df['Newspaper'].value_counts())

Wall Street Journal    168874
New York Times         164626
Los Angeles Times       78438
The Washington Post     72705
Chicago Tribune         56154
Boston Globe            44862
USA Today               22513
Name: Newspaper, dtype: int64


In [11]:
# Revised function to estimate index (suppressing constant)
def estimate_index(var_name):
    FE_OLS=sm.ols(formula=var_name + ' ~ 0+C(YM)+C(Newspaper)',
        data=df).fit()
    print(FE_OLS.summary())

    FE_estimates=pd.DataFrame()
    FE_estimates[var_name+'Index']=FE_OLS.params[0:max(df_ym['YM'])]
    FE_estimates=FE_estimates.reset_index().rename(columns={'index':'FE'})
    FE_estimates['YM']=FE_estimates['FE'].str.split("[",expand=True)[1].str.split("]",expand=True)[0].astype('int64')
    
    return FE_estimates

In [12]:
# Uncertainty index
UncertaintyIndex=estimate_index('UncertaintyScore')

                            OLS Regression Results                            
Dep. Variable:       UncertaintyScore   R-squared:                       0.005
Model:                            OLS   Adj. R-squared:                  0.004
Method:                 Least Squares   F-statistic:                     7.071
Date:                Tue, 25 Jan 2022   Prob (F-statistic):               0.00
Time:                        15:51:41   Log-Likelihood:            -8.2907e+05
No. Observations:              608172   AIC:                         1.659e+06
Df Residuals:                  607722   BIC:                         1.664e+06
Df Model:                         449                                         
Covariance Type:            nonrobust                                         
                                          coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------
C(

In [13]:
# LM index
LMindex=estimate_index('LMscore')

                            OLS Regression Results                            
Dep. Variable:                LMscore   R-squared:                       0.006
Model:                            OLS   Adj. R-squared:                  0.005
Method:                 Least Squares   F-statistic:                     7.700
Date:                Tue, 25 Jan 2022   Prob (F-statistic):               0.00
Time:                        15:52:25   Log-Likelihood:            -1.4329e+06
No. Observations:              608172   AIC:                         2.867e+06
Df Residuals:                  607722   BIC:                         2.872e+06
Df Model:                         449                                         
Covariance Type:            nonrobust                                         
                                          coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------
C(

In [14]:
# GI index
GIindex=estimate_index('GIscore')

                            OLS Regression Results                            
Dep. Variable:                GIscore   R-squared:                       0.005
Model:                            OLS   Adj. R-squared:                  0.005
Method:                 Least Squares   F-statistic:                     7.183
Date:                Tue, 25 Jan 2022   Prob (F-statistic):               0.00
Time:                        15:53:10   Log-Likelihood:            -1.7004e+06
No. Observations:              608172   AIC:                         3.402e+06
Df Residuals:                  607722   BIC:                         3.407e+06
Df Model:                         449                                         
Covariance Type:            nonrobust                                         
                                          coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------
C(

In [15]:
# LSD index
LSDindex=estimate_index('LSDscore')

                            OLS Regression Results                            
Dep. Variable:               LSDscore   R-squared:                       0.006
Model:                            OLS   Adj. R-squared:                  0.005
Method:                 Least Squares   F-statistic:                     8.147
Date:                Tue, 25 Jan 2022   Prob (F-statistic):               0.00
Time:                        15:53:53   Log-Likelihood:            -1.6071e+06
No. Observations:              608172   AIC:                         3.215e+06
Df Residuals:                  607722   BIC:                         3.220e+06
Df Model:                         449                                         
Covariance Type:            nonrobust                                         
                                          coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------
C(

In [16]:
# Merge indexes
sentimentIndex=df_ym.merge(UncertaintyIndex,on='YM',how='outer').\
        merge(LMindex,on='YM',how='outer').\
        merge(GIindex,on='YM',how='outer').\
        merge(LSDindex,on='YM',how='outer').\
        sort_values(['Year','Month'])
print(sentimentIndex.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 444 entries, 0 to 443
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Year                   444 non-null    int64  
 1   Month                  444 non-null    int64  
 2   YM                     444 non-null    int64  
 3   FE_x                   444 non-null    object 
 4   UncertaintyScoreIndex  444 non-null    float64
 5   FE_y                   444 non-null    object 
 6   LMscoreIndex           444 non-null    float64
 7   FE_x                   444 non-null    object 
 8   GIscoreIndex           444 non-null    float64
 9   FE_y                   444 non-null    object 
 10  LSDscoreIndex          444 non-null    float64
dtypes: float64(4), int64(3), object(4)
memory usage: 41.6+ KB
None


In [17]:
sentimentIndex=sentimentIndex.drop(['FE_x','FE_y'],axis=1).\
        rename(columns={'UncertaintyScoreIndex':'UncertaintyIndex','LMscoreIndex':'LMindex',
                        'GIscoreIndex':'GIindex','LSDscoreIndex':'LSDindex'})

In [18]:
print(sentimentIndex.head())

   Year  Month  YM  UncertaintyIndex   LMindex   GIindex  LSDindex
0  1985      1   1          0.605388 -1.802030  1.355412  0.260553
1  1985      2   2          0.592809 -1.926507  1.461178  0.051100
2  1985      3   3          0.628167 -1.975556  1.162074 -0.074685
3  1985      4   4          0.652404 -1.867413  1.406196  0.078924
4  1985      5   5          0.652003 -1.798869  1.442610  0.273726


In [19]:
print(sentimentIndex.info())
print(sentimentIndex.tail())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 444 entries, 0 to 443
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Year              444 non-null    int64  
 1   Month             444 non-null    int64  
 2   YM                444 non-null    int64  
 3   UncertaintyIndex  444 non-null    float64
 4   LMindex           444 non-null    float64
 5   GIindex           444 non-null    float64
 6   LSDindex          444 non-null    float64
dtypes: float64(4), int64(3)
memory usage: 27.8 KB
None
     Year  Month   YM  UncertaintyIndex   LMindex   GIindex  LSDindex
439  2021      8  440          0.754957 -1.716133  1.427658  0.365124
440  2021      9  441          0.728919 -1.867266  1.468827  0.188857
441  2021     10  442          0.764055 -1.731653  1.725515  0.387991
442  2021     11  443          0.685611 -1.741443  1.583064  0.329735
443  2021     12  444          0.778421 -1.855297  1.442019  0.1959

In [20]:
# Export
sentimentIndex.to_csv('/home/ec2-user/SageMaker/New Uncertainty/Jan1985-Dec2021/RegRelevant_MonthlySentimentIndex_'+str(end_month)+'.csv',index=False)